# Module 10: 性能優化與部署

## 學習目標
- 掌握 DataLoader 優化技巧
- 學會混合精度訓練加速
- 理解梯度累積處理大批次
- 模型保存、載入與部署

## 為什麼需要優化？

在實際應用中，訓練效率和部署品質直接影響：
- 訓練成本（時間和 GPU 費用）
- 產品迭代速度
- 線上服務的延遲和吞吐量

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
import time
import os
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, Any

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

---
## Part 1: DataLoader 優化

### 資料載入瓶頸

訓練過程中，GPU 可能在等待 CPU 準備資料：

```
未優化：
CPU: [載入資料] ──► [等待] ──► [載入資料] ──► [等待]
GPU: [等待] ──► [訓練] ──► [等待] ──► [訓練]

優化後：
CPU: [載入批次 1] [載入批次 2] [載入批次 3] ...
GPU: ──────────► [訓練 1] [訓練 2] [訓練 3] ...
```

In [ ]:
# 準備資料
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)

def benchmark_dataloader(dataset, **kwargs):
    """
    測試 DataLoader 的載入速度
    """
    loader = DataLoader(dataset, **kwargs)
    
    start = time.time()
    n_batches = 0
    
    for batch in loader:
        n_batches += 1
        if n_batches >= 100:  # 測試 100 個批次
            break
    
    elapsed = time.time() - start
    batches_per_sec = n_batches / elapsed
    
    return batches_per_sec, elapsed

# 測試不同配置
configs = [
    {'batch_size': 64, 'num_workers': 0, 'pin_memory': False},
    {'batch_size': 64, 'num_workers': 2, 'pin_memory': False},
    {'batch_size': 64, 'num_workers': 4, 'pin_memory': False},
    {'batch_size': 64, 'num_workers': 4, 'pin_memory': True},
]

print("DataLoader 配置比較：")
print("="*60)

results = []
for config in configs:
    bps, elapsed = benchmark_dataloader(train_dataset, **config)
    results.append(bps)
    print(f"workers={config['num_workers']}, pin_memory={config['pin_memory']}")
    print(f"  速度: {bps:.1f} batches/sec")
    
print(f"\n最快配置比最慢快 {max(results)/min(results):.1f}x")

### DataLoader 最佳實踐

In [ ]:
def create_optimized_dataloader(
    dataset,
    batch_size=64,
    shuffle=True,
    num_workers=None,
    pin_memory=True,
    prefetch_factor=2,
    persistent_workers=True
):
    """
    建立優化的 DataLoader
    
    參數說明：
    - num_workers: 並行資料載入的進程數
      建議設為 CPU 核心數的一半或 4-8
      
    - pin_memory: 將資料固定在 CPU 記憶體
      加速 CPU→GPU 傳輸，建議開啟
      
    - prefetch_factor: 每個 worker 預取的批次數
      增加可以減少等待，但佔用更多記憶體
      
    - persistent_workers: 保持 workers 存活
      避免每個 epoch 重新啟動進程的開銷
    """
    
    # 自動決定 num_workers
    if num_workers is None:
        num_workers = min(4, os.cpu_count() or 1)
    
    # 如果 num_workers=0，某些參數需要調整
    if num_workers == 0:
        prefetch_factor = None
        persistent_workers = False
    
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory and torch.cuda.is_available(),
        prefetch_factor=prefetch_factor,
        persistent_workers=persistent_workers and num_workers > 0,
        drop_last=True,  # 避免最後一個小批次影響 BatchNorm
    )
    
    return loader

# 建立優化的 DataLoader
train_loader = create_optimized_dataloader(train_dataset, batch_size=128)
print(f"優化的 DataLoader 已建立")
print(f"  batch_size: 128")
print(f"  num_workers: {train_loader.num_workers}")
print(f"  pin_memory: {train_loader.pin_memory}")

### 自訂 Dataset 優化

In [ ]:
class OptimizedDataset(Dataset):
    """
    優化的自訂 Dataset
    
    優化技巧：
    1. 預載入小資料集到記憶體
    2. 使用記憶體映射處理大檔案
    3. 在 __getitem__ 中避免重複計算
    """
    
    def __init__(self, data, labels, transform=None, preload=True):
        """
        Args:
            data: 資料（numpy array 或檔案路徑列表）
            labels: 標籤
            transform: 資料轉換
            preload: 是否預載入到記憶體
        """
        self.labels = labels
        self.transform = transform
        
        if preload:
            # 預載入到記憶體
            if isinstance(data, str):
                self.data = np.load(data)
            else:
                self.data = np.array(data)
        else:
            # 使用記憶體映射（適合大檔案）
            self.data = np.load(data, mmap_mode='r')
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # 避免在這裡做複雜計算
        x = self.data[idx]
        y = self.labels[idx]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y

print("OptimizedDataset 類別已定義")

---
## Part 2: 混合精度訓練 (AMP)

### 什麼是混合精度？

```
精度類型：
FP32 (32 位)：標準精度，4 bytes/參數
FP16 (16 位)：半精度，2 bytes/參數
BF16 (16 位)：Brain Float，2 bytes/參數（更好的數值範圍）

混合精度策略：
- 前向傳播：使用 FP16/BF16（速度快）
- 權重更新：使用 FP32（精度高）
- 損失縮放：防止梯度下溢

好處：
- 訓練速度提升 2-3x
- 記憶體使用減半
- 可以訓練更大的模型/批次
```

In [ ]:
from torch.cuda.amp import autocast, GradScaler

class AMPTrainer:
    """
    自動混合精度訓練器
    """
    
    def __init__(self, model, optimizer, criterion, use_amp=True, amp_dtype=torch.float16):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.use_amp = use_amp and torch.cuda.is_available()
        self.amp_dtype = amp_dtype
        
        # 梯度縮放器（防止 FP16 梯度下溢）
        self.scaler = GradScaler() if self.use_amp else None
        
    def train_step(self, x, y):
        """
        單步訓練
        """
        self.optimizer.zero_grad()
        
        if self.use_amp:
            # 混合精度前向傳播
            with autocast(dtype=self.amp_dtype):
                output = self.model(x)
                loss = self.criterion(output, y)
            
            # 縮放損失並反向傳播
            self.scaler.scale(loss).backward()
            
            # 更新權重（自動處理縮放）
            self.scaler.step(self.optimizer)
            self.scaler.update()
        else:
            # 標準訓練
            output = self.model(x)
            loss = self.criterion(output, y)
            loss.backward()
            self.optimizer.step()
        
        return loss.item()

# 比較 AMP 和標準訓練
def compare_training_speed():
    """
    比較有無 AMP 的訓練速度
    """
    # 建立一個較大的模型
    model = nn.Sequential(
        nn.Conv2d(1, 64, 3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 128, 3, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d(1),
        nn.Flatten(),
        nn.Linear(128, 10)
    ).to(device)
    
    results = {}
    
    for use_amp in [False, True]:
        # 重置模型
        model_copy = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, 10)
        ).to(device)
        
        optimizer = torch.optim.Adam(model_copy.parameters(), lr=1e-3)
        criterion = nn.CrossEntropyLoss()
        trainer = AMPTrainer(model_copy, optimizer, criterion, use_amp=use_amp)
        
        # 預熱
        x = torch.randn(64, 1, 28, 28).to(device)
        y = torch.randint(0, 10, (64,)).to(device)
        for _ in range(10):
            trainer.train_step(x, y)
        
        # 計時
        torch.cuda.synchronize()
        start = time.time()
        
        for _ in range(100):
            trainer.train_step(x, y)
        
        torch.cuda.synchronize()
        elapsed = time.time() - start
        
        mode = "AMP" if use_amp else "FP32"
        results[mode] = elapsed
        print(f"{mode}: {elapsed:.3f}s for 100 steps")
    
    speedup = results['FP32'] / results['AMP']
    print(f"\nAMP 加速: {speedup:.2f}x")

if torch.cuda.is_available():
    compare_training_speed()
else:
    print("需要 CUDA 來比較 AMP 性能")

---
## Part 3: 梯度累積

當 VRAM 不足以容納大批次時，使用梯度累積模擬。

In [ ]:
class GradientAccumulationTrainer:
    """
    支援梯度累積的訓練器
    
    有效批次大小 = micro_batch_size × accumulation_steps
    """
    
    def __init__(
        self, 
        model, 
        optimizer, 
        criterion,
        accumulation_steps=4,
        use_amp=True,
        max_grad_norm=1.0  # 梯度裁剪
    ):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.accumulation_steps = accumulation_steps
        self.use_amp = use_amp and torch.cuda.is_available()
        self.max_grad_norm = max_grad_norm
        
        self.scaler = GradScaler() if self.use_amp else None
        self.current_step = 0
        
    def train_step(self, x, y):
        """
        單步訓練（可能不會立即更新權重）
        """
        # 計算損失
        if self.use_amp:
            with autocast(dtype=torch.float16):
                output = self.model(x)
                # 重要：除以累積步數！
                loss = self.criterion(output, y) / self.accumulation_steps
            self.scaler.scale(loss).backward()
        else:
            output = self.model(x)
            loss = self.criterion(output, y) / self.accumulation_steps
            loss.backward()
        
        self.current_step += 1
        
        # 每 accumulation_steps 步更新一次
        if self.current_step % self.accumulation_steps == 0:
            if self.use_amp:
                # 解除縮放梯度
                self.scaler.unscale_(self.optimizer)
                
                # 梯度裁剪
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.max_grad_norm)
                
                # 更新
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.max_grad_norm)
                self.optimizer.step()
            
            self.optimizer.zero_grad()
            return True, loss.item() * self.accumulation_steps
        
        return False, loss.item() * self.accumulation_steps

# 示範
print("梯度累積示範：")
print("假設 GPU 只能處理 batch_size=8，但想要有效 batch_size=32")
print("設定 accumulation_steps=4")
print("\n每 4 個微批次後更新一次權重")

---
## Part 4: 模型保存與載入

### 保存選項

In [ ]:
class ModelSaver:
    """
    模型保存工具
    
    支援：
    - 保存權重
    - 保存完整模型
    - 保存訓練檢查點（包含優化器狀態）
    """
    
    @staticmethod
    def save_weights(model, path):
        """
        只保存權重（推薦，檔案小）
        """
        torch.save(model.state_dict(), path)
        print(f"權重已保存至: {path}")
    
    @staticmethod
    def load_weights(model, path, strict=True):
        """
        載入權重
        
        Args:
            strict: 是否要求所有鍵都匹配
        """
        model.load_state_dict(torch.load(path), strict=strict)
        print(f"權重已從 {path} 載入")
        return model
    
    @staticmethod
    def save_checkpoint(
        model, 
        optimizer, 
        epoch, 
        loss, 
        path,
        scheduler=None,
        extra_info=None
    ):
        """
        保存完整訓練檢查點（用於恢復訓練）
        """
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }
        
        if scheduler is not None:
            checkpoint['scheduler_state_dict'] = scheduler.state_dict()
        
        if extra_info is not None:
            checkpoint['extra_info'] = extra_info
        
        torch.save(checkpoint, path)
        print(f"檢查點已保存至: {path}")
    
    @staticmethod
    def load_checkpoint(path, model, optimizer=None, scheduler=None):
        """
        載入訓練檢查點
        """
        checkpoint = torch.load(path)
        
        model.load_state_dict(checkpoint['model_state_dict'])
        
        if optimizer is not None and 'optimizer_state_dict' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        if scheduler is not None and 'scheduler_state_dict' in checkpoint:
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
        print(f"檢查點已從 {path} 載入")
        print(f"  Epoch: {checkpoint['epoch']}")
        print(f"  Loss: {checkpoint['loss']:.4f}")
        
        return checkpoint

# 示範
demo_model = nn.Linear(10, 2)
demo_optimizer = torch.optim.Adam(demo_model.parameters())

# 保存
ModelSaver.save_weights(demo_model, './demo_weights.pt')
ModelSaver.save_checkpoint(demo_model, demo_optimizer, epoch=10, loss=0.5, path='./demo_checkpoint.pt')

# 清理
import os
os.remove('./demo_weights.pt')
os.remove('./demo_checkpoint.pt')

### 模型導出（用於部署）

In [ ]:
class ModelExporter:
    """
    模型導出工具
    
    支援格式：
    - TorchScript (推薦，PyTorch 原生)
    - ONNX (跨平台)
    """
    
    @staticmethod
    def to_torchscript(model, example_input, path, method='trace'):
        """
        導出為 TorchScript
        
        Args:
            method: 'trace' 或 'script'
            - trace: 追蹤一次前向傳播，適合固定結構
            - script: 編譯程式碼，支援控制流
        """
        model.eval()
        
        if method == 'trace':
            scripted = torch.jit.trace(model, example_input)
        else:
            scripted = torch.jit.script(model)
        
        scripted.save(path)
        print(f"TorchScript 模型已保存至: {path}")
        
        return scripted
    
    @staticmethod
    def to_onnx(
        model, 
        example_input, 
        path,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes=None
    ):
        """
        導出為 ONNX 格式
        
        ONNX 可以在多種推理引擎上運行：
        - ONNX Runtime
        - TensorRT
        - OpenVINO
        """
        model.eval()
        
        if dynamic_axes is None:
            # 預設：batch size 動態
            dynamic_axes = {
                'input': {0: 'batch_size'},
                'output': {0: 'batch_size'}
            }
        
        torch.onnx.export(
            model,
            example_input,
            path,
            input_names=input_names,
            output_names=output_names,
            dynamic_axes=dynamic_axes,
            opset_version=14,
        )
        
        print(f"ONNX 模型已保存至: {path}")
    
    @staticmethod
    def load_torchscript(path):
        """
        載入 TorchScript 模型
        """
        model = torch.jit.load(path)
        print(f"TorchScript 模型已從 {path} 載入")
        return model

# 示範 TorchScript 導出
demo_model = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

example_input = torch.randn(1, 784)
scripted = ModelExporter.to_torchscript(demo_model, example_input, './demo_model.pt')

# 驗證輸出一致
with torch.no_grad():
    original_out = demo_model(example_input)
    scripted_out = scripted(example_input)
    diff = (original_out - scripted_out).abs().max().item()
    print(f"原始 vs TorchScript 輸出差異: {diff:.10f}")

# 清理
os.remove('./demo_model.pt')

---
## Part 5: 推理優化

In [ ]:
class InferenceOptimizer:
    """
    推理優化工具
    """
    
    @staticmethod
    def optimize_model_for_inference(model):
        """
        為推理優化模型
        """
        model.eval()  # 評估模式
        
        # 凍結 BatchNorm（如果有）
        for module in model.modules():
            if isinstance(module, nn.BatchNorm2d) or isinstance(module, nn.BatchNorm1d):
                module.eval()
                module.track_running_stats = False
        
        return model
    
    @staticmethod
    def fuse_conv_bn(model):
        """
        融合 Conv + BatchNorm 層（加速推理）
        
        數學原理：
        BN(Conv(x)) = gamma * (Conv(x) - mean) / sqrt(var + eps) + beta
        可以合併為一個等效的 Conv
        """
        # PyTorch 提供的融合工具
        try:
            from torch.quantization import fuse_modules
            # 需要知道具體的模組名稱來融合
            print("使用 torch.quantization.fuse_modules 進行融合")
        except ImportError:
            print("融合功能需要 PyTorch >= 1.3")
        
        return model
    
    @staticmethod
    @torch.no_grad()
    def benchmark_inference(model, input_shape, n_warmup=10, n_runs=100):
        """
        測試推理速度
        """
        model.eval()
        device = next(model.parameters()).device
        
        x = torch.randn(*input_shape).to(device)
        
        # 預熱
        for _ in range(n_warmup):
            _ = model(x)
        
        # 同步（確保 GPU 完成）
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        # 計時
        start = time.time()
        for _ in range(n_runs):
            _ = model(x)
        
        if device.type == 'cuda':
            torch.cuda.synchronize()
        
        elapsed = time.time() - start
        
        avg_time = elapsed / n_runs * 1000  # ms
        throughput = n_runs * input_shape[0] / elapsed  # samples/sec
        
        print(f"推理性能:")
        print(f"  平均延遲: {avg_time:.2f} ms")
        print(f"  吞吐量: {throughput:.0f} samples/sec")
        
        return {'latency_ms': avg_time, 'throughput': throughput}

# 測試
test_model = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
    nn.Linear(64, 10)
).to(device)

InferenceOptimizer.benchmark_inference(test_model, (32, 1, 28, 28))

---
## Part 6: 簡單 API 部署

使用 FastAPI 建立簡單的模型推理服務。

In [ ]:
# FastAPI 推理服務範例

fastapi_server_code = '''
# app.py - FastAPI 推理服務

from fastapi import FastAPI, File, UploadFile
from pydantic import BaseModel
import torch
import torch.nn.functional as F
from PIL import Image
import io
import numpy as np
from typing import List

app = FastAPI(title="MNIST 分類服務")

# 載入模型（啟動時載入一次）
model = torch.jit.load("model.pt")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

class PredictionResponse(BaseModel):
    predicted_class: int
    confidence: float
    probabilities: List[float]

@app.get("/")
def root():
    return {"message": "MNIST 分類 API"}

@app.get("/health")
def health_check():
    return {"status": "healthy", "device": str(device)}

@app.post("/predict", response_model=PredictionResponse)
async def predict(file: UploadFile = File(...)):
    """
    接收圖像並返回預測結果
    """
    # 讀取並預處理圖像
    contents = await file.read()
    image = Image.open(io.BytesIO(contents)).convert("L")  # 轉灰階
    image = image.resize((28, 28))
    
    # 轉為張量
    img_array = np.array(image, dtype=np.float32) / 255.0
    img_tensor = torch.from_numpy(img_array).unsqueeze(0).unsqueeze(0)  # [1, 1, 28, 28]
    img_tensor = (img_tensor - 0.5) / 0.5  # 歸一化
    img_tensor = img_tensor.to(device)
    
    # 推理
    with torch.no_grad():
        output = model(img_tensor)
        probs = F.softmax(output, dim=1)
        confidence, predicted = probs.max(dim=1)
    
    return PredictionResponse(
        predicted_class=predicted.item(),
        confidence=confidence.item(),
        probabilities=probs[0].tolist()
    )

# 批次預測
@app.post("/predict_batch")
async def predict_batch(files: List[UploadFile] = File(...)):
    """
    批次預測（更高效）
    """
    tensors = []
    
    for file in files:
        contents = await file.read()
        image = Image.open(io.BytesIO(contents)).convert("L")
        image = image.resize((28, 28))
        img_array = np.array(image, dtype=np.float32) / 255.0
        img_tensor = torch.from_numpy(img_array)
        tensors.append(img_tensor)
    
    # 批次處理
    batch = torch.stack(tensors).unsqueeze(1)  # [B, 1, 28, 28]
    batch = (batch - 0.5) / 0.5
    batch = batch.to(device)
    
    with torch.no_grad():
        outputs = model(batch)
        probs = F.softmax(outputs, dim=1)
        confidences, predictions = probs.max(dim=1)
    
    return {
        "predictions": predictions.tolist(),
        "confidences": confidences.tolist()
    }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

print("FastAPI 推理服務範例：")
print("="*60)
print(fastapi_server_code)

In [ ]:
# Docker 部署配置

dockerfile_content = '''
# Dockerfile
FROM python:3.10-slim

WORKDIR /app

# 安裝依賴
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# 複製模型和程式碼
COPY model.pt .
COPY app.py .

# 暴露端口
EXPOSE 8000

# 啟動服務
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
'''

requirements_content = '''
# requirements.txt
torch>=2.0.0
fastapi>=0.100.0
uvicorn>=0.23.0
python-multipart>=0.0.6
pillow>=10.0.0
numpy>=1.24.0
'''

print("Dockerfile:")
print(dockerfile_content)
print("\nrequirements.txt:")
print(requirements_content)

---
## 練習題

### 練習 1: 完整訓練管線

In [ ]:
# 練習 1: 整合所有優化的訓練管線

class OptimizedTrainingPipeline:
    """
    整合所有優化技術的訓練管線
    """
    
    def __init__(
        self,
        model,
        train_dataset,
        val_dataset=None,
        batch_size=64,
        accumulation_steps=1,
        use_amp=True,
        lr=1e-3,
        weight_decay=1e-4,
        checkpoint_dir='./checkpoints',
        save_every=5,
    ):
        self.model = model.to(device)
        self.batch_size = batch_size
        self.accumulation_steps = accumulation_steps
        self.use_amp = use_amp and torch.cuda.is_available()
        self.checkpoint_dir = checkpoint_dir
        self.save_every = save_every
        
        # 建立優化的 DataLoader
        self.train_loader = create_optimized_dataloader(
            train_dataset, batch_size=batch_size, shuffle=True
        )
        
        if val_dataset is not None:
            self.val_loader = create_optimized_dataloader(
                val_dataset, batch_size=batch_size * 2, shuffle=False
            )
        else:
            self.val_loader = None
        
        # 優化器和調度器
        self.optimizer = torch.optim.AdamW(
            model.parameters(), lr=lr, weight_decay=weight_decay
        )
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=100
        )
        
        # 損失函數
        self.criterion = nn.CrossEntropyLoss()
        
        # AMP
        self.scaler = GradScaler() if self.use_amp else None
        
        # 歷史記錄
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'val_acc': [],
            'lr': []
        }
        
        # 建立檢查點目錄
        os.makedirs(checkpoint_dir, exist_ok=True)
        
    def train_epoch(self):
        self.model.train()
        total_loss = 0
        n_batches = 0
        
        self.optimizer.zero_grad()
        
        for batch_idx, (data, target) in enumerate(self.train_loader):
            data, target = data.to(device), target.to(device)
            
            if self.use_amp:
                with autocast(dtype=torch.float16):
                    output = self.model(data)
                    loss = self.criterion(output, target) / self.accumulation_steps
                self.scaler.scale(loss).backward()
            else:
                output = self.model(data)
                loss = self.criterion(output, target) / self.accumulation_steps
                loss.backward()
            
            if (batch_idx + 1) % self.accumulation_steps == 0:
                if self.use_amp:
                    self.scaler.step(self.optimizer)
                    self.scaler.update()
                else:
                    self.optimizer.step()
                self.optimizer.zero_grad()
            
            total_loss += loss.item() * self.accumulation_steps
            n_batches += 1
        
        return total_loss / n_batches
    
    @torch.no_grad()
    def validate(self):
        if self.val_loader is None:
            return None, None
        
        self.model.eval()
        total_loss = 0
        correct = 0
        total = 0
        
        for data, target in self.val_loader:
            data, target = data.to(device), target.to(device)
            
            output = self.model(data)
            loss = self.criterion(output, target)
            
            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += (pred == target).sum().item()
            total += target.size(0)
        
        return total_loss / len(self.val_loader), correct / total
    
    def train(self, epochs):
        best_val_acc = 0
        
        for epoch in range(epochs):
            start = time.time()
            
            # 訓練
            train_loss = self.train_epoch()
            
            # 驗證
            val_loss, val_acc = self.validate()
            
            # 更新學習率
            self.scheduler.step()
            current_lr = self.scheduler.get_last_lr()[0]
            
            # 記錄歷史
            self.history['train_loss'].append(train_loss)
            self.history['lr'].append(current_lr)
            if val_loss is not None:
                self.history['val_loss'].append(val_loss)
                self.history['val_acc'].append(val_acc)
            
            elapsed = time.time() - start
            
            # 列印進度
            print(f"Epoch {epoch+1}/{epochs} ({elapsed:.1f}s)")
            print(f"  Train Loss: {train_loss:.4f}")
            if val_loss is not None:
                print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            print(f"  LR: {current_lr:.6f}")
            
            # 保存檢查點
            if (epoch + 1) % self.save_every == 0:
                path = os.path.join(self.checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pt')
                ModelSaver.save_checkpoint(
                    self.model, self.optimizer, epoch+1, train_loss, path,
                    scheduler=self.scheduler
                )
            
            # 保存最佳模型
            if val_acc is not None and val_acc > best_val_acc:
                best_val_acc = val_acc
                path = os.path.join(self.checkpoint_dir, 'best_model.pt')
                ModelSaver.save_weights(self.model, path)
                print(f"  新最佳模型! Acc: {val_acc:.4f}")
        
        return self.history

# 使用範例
print("OptimizedTrainingPipeline 類別已定義")
print("\n使用範例：")
print('''
model = YourModel()
pipeline = OptimizedTrainingPipeline(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    batch_size=64,
    accumulation_steps=4,  # 有效 batch = 256
    use_amp=True,
    lr=1e-3,
)
history = pipeline.train(epochs=50)
''')

### 練習 2: 模型效能分析

In [ ]:
# 練習 2: 模型效能分析工具

class ModelProfiler:
    """
    模型效能分析工具
    """
    
    @staticmethod
    def count_parameters(model):
        """
        計算模型參數量
        """
        total = sum(p.numel() for p in model.parameters())
        trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
        
        print(f"參數統計：")
        print(f"  總參數量: {total:,}")
        print(f"  可訓練參數: {trainable:,}")
        print(f"  凍結參數: {total - trainable:,}")
        print(f"  模型大小: ~{total * 4 / 1024**2:.1f} MB (FP32)")
        
        return total, trainable
    
    @staticmethod
    def estimate_memory(model, input_shape, batch_size=1):
        """
        估算模型記憶體使用
        """
        # 參數記憶體
        param_mem = sum(p.numel() * p.element_size() for p in model.parameters())
        
        # 估算激活值記憶體（粗略）
        # 需要實際運行來獲得準確值
        
        print(f"記憶體估算（batch_size={batch_size}）：")
        print(f"  參數: {param_mem / 1024**2:.1f} MB")
        
        return param_mem
    
    @staticmethod
    def layer_summary(model, input_shape):
        """
        打印每層的輸出形狀和參數量
        """
        def hook_fn(module, input, output):
            if hasattr(output, 'shape'):
                summary.append({
                    'name': module.__class__.__name__,
                    'output_shape': list(output.shape),
                    'params': sum(p.numel() for p in module.parameters())
                })
        
        summary = []
        hooks = []
        
        for module in model.modules():
            if len(list(module.children())) == 0:  # 只對葉子節點
                hooks.append(module.register_forward_hook(hook_fn))
        
        # 運行一次前向傳播
        x = torch.randn(*input_shape).to(device)
        model.eval()
        with torch.no_grad():
            model(x)
        
        # 移除 hooks
        for hook in hooks:
            hook.remove()
        
        # 打印摘要
        print(f"{'Layer':<30} {'Output Shape':<20} {'Params':>10}")
        print("="*62)
        
        total_params = 0
        for layer in summary:
            print(f"{layer['name']:<30} {str(layer['output_shape']):<20} {layer['params']:>10,}")
            total_params += layer['params']
        
        print("="*62)
        print(f"{'Total':<30} {'':<20} {total_params:>10,}")
        
        return summary

# 測試
test_model = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(64 * 7 * 7, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
).to(device)

ModelProfiler.count_parameters(test_model)
print()
ModelProfiler.layer_summary(test_model, (1, 1, 28, 28))

### 練習 3: 學習率尋找器

In [ ]:
# 練習 3: 學習率尋找器

class LRFinder:
    """
    學習率尋找器
    
    通過逐漸增加學習率來找到最佳學習率範圍
    """
    
    def __init__(self, model, optimizer, criterion, device):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.device = device
        
        # 保存初始狀態
        self.model_state = model.state_dict()
        self.optimizer_state = optimizer.state_dict()
        
    def find(
        self, 
        train_loader, 
        start_lr=1e-7, 
        end_lr=10, 
        num_iter=100,
        smooth_factor=0.05
    ):
        """
        執行學習率搜索
        """
        # 計算每步的學習率乘數
        lr_mult = (end_lr / start_lr) ** (1 / num_iter)
        lr = start_lr
        
        # 設置初始學習率
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        
        # 記錄
        lrs = []
        losses = []
        avg_loss = 0
        best_loss = float('inf')
        
        self.model.train()
        iterator = iter(train_loader)
        
        for i in range(num_iter):
            try:
                data, target = next(iterator)
            except StopIteration:
                iterator = iter(train_loader)
                data, target = next(iterator)
            
            data, target = data.to(self.device), target.to(self.device)
            
            # 前向傳播
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = self.criterion(output, target)
            
            # 平滑損失
            avg_loss = smooth_factor * loss.item() + (1 - smooth_factor) * avg_loss
            smoothed_loss = avg_loss / (1 - smooth_factor ** (i + 1))
            
            # 記錄
            lrs.append(lr)
            losses.append(smoothed_loss)
            
            # 如果損失爆炸，停止
            if smoothed_loss > 4 * best_loss:
                print(f"損失爆炸，停止於 lr={lr:.2e}")
                break
            
            if smoothed_loss < best_loss:
                best_loss = smoothed_loss
            
            # 反向傳播
            loss.backward()
            self.optimizer.step()
            
            # 更新學習率
            lr *= lr_mult
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = lr
        
        # 恢復初始狀態
        self.model.load_state_dict(self.model_state)
        self.optimizer.load_state_dict(self.optimizer_state)
        
        return lrs, losses
    
    def plot(self, lrs, losses, suggest=True):
        """
        繪製學習率 vs 損失曲線
        """
        plt.figure(figsize=(10, 4))
        plt.plot(lrs, losses)
        plt.xscale('log')
        plt.xlabel('Learning Rate')
        plt.ylabel('Loss')
        plt.title('Learning Rate Finder')
        
        if suggest:
            # 找到損失下降最快的點
            gradients = np.gradient(losses)
            min_grad_idx = np.argmin(gradients)
            suggested_lr = lrs[min_grad_idx]
            
            plt.axvline(suggested_lr, color='r', linestyle='--', 
                       label=f'建議 LR: {suggested_lr:.2e}')
            plt.legend()
            
            print(f"建議學習率: {suggested_lr:.2e}")
        
        plt.grid(True, alpha=0.3)
        plt.show()

# 使用範例
print("LRFinder 使用範例：")
print('''
model = YourModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-7)
criterion = nn.CrossEntropyLoss()

lr_finder = LRFinder(model, optimizer, criterion, device)
lrs, losses = lr_finder.find(train_loader)
lr_finder.plot(lrs, losses)
''')

---
## Part 7: 進階優化技巧

### 7.1 PyTorch 2.0 torch.compile()

PyTorch 2.0 引入了 `torch.compile()`，可以自動優化模型的執行圖。

In [ ]:
# ========== PyTorch 2.0 torch.compile() ==========

def demonstrate_torch_compile():
    """
    展示 torch.compile() 的使用和性能提升
    """
    # 建立測試模型
    class TestModel(nn.Module):
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
            self.bn1 = nn.BatchNorm2d(64)
            self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
            self.bn2 = nn.BatchNorm2d(128)
            self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
            self.bn3 = nn.BatchNorm2d(256)
            self.pool = nn.AdaptiveAvgPool2d(1)
            self.fc = nn.Linear(256, 10)
        
        def forward(self, x):
            x = F.relu(self.bn1(self.conv1(x)))
            x = F.relu(self.bn2(self.conv2(x)))
            x = F.relu(self.bn3(self.conv3(x)))
            x = self.pool(x).flatten(1)
            return self.fc(x)
    
    model = TestModel().to(device)
    x = torch.randn(32, 3, 64, 64).to(device)
    
    # 原始模型測試
    model.eval()
    
    # 預熱
    for _ in range(10):
        _ = model(x)
    
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    start = time.time()
    for _ in range(100):
        _ = model(x)
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    eager_time = time.time() - start
    
    # 編譯模型
    try:
        compiled_model = torch.compile(model, mode="reduce-overhead")
        
        # 預熱（編譯發生在第一次前向傳播）
        for _ in range(10):
            _ = compiled_model(x)
        
        torch.cuda.synchronize() if torch.cuda.is_available() else None
        start = time.time()
        for _ in range(100):
            _ = compiled_model(x)
        torch.cuda.synchronize() if torch.cuda.is_available() else None
        compiled_time = time.time() - start
        
        print("torch.compile() 性能比較：")
        print(f"  原始模型: {eager_time*10:.2f} ms/100 batches")
        print(f"  編譯模型: {compiled_time*10:.2f} ms/100 batches")
        print(f"  加速比: {eager_time/compiled_time:.2f}x")
        
    except Exception as e:
        print(f"torch.compile() 需要 PyTorch 2.0+")
        print(f"當前版本: {torch.__version__}")

# 執行測試
print("PyTorch 版本:", torch.__version__)
demonstrate_torch_compile()

print("""
💡 torch.compile() 模式說明：
- 'default': 平衡編譯時間和加速
- 'reduce-overhead': 最大化運行時加速（推薦 GPU）
- 'max-autotune': 最大優化（編譯時間長）
""")

### 7.2 模型量化 (Quantization)

量化可以將模型從 FP32 壓縮到 INT8，減少模型大小和推理時間。

```
量化類型：
1. 動態量化 (Dynamic Quantization)
   - 權重量化為 INT8
   - 激活在運行時量化
   - 簡單易用，適合 CPU 部署

2. 靜態量化 (Static Quantization)
   - 權重和激活都量化
   - 需要校準數據
   - 更高的壓縮率

3. 量化感知訓練 (QAT)
   - 在訓練時模擬量化
   - 精度損失最小
   - 需要重新訓練
```

In [ ]:
# ========== 模型量化 ==========

class ModelQuantizer:
    """
    模型量化工具
    """
    
    @staticmethod
    def dynamic_quantize(model):
        """
        動態量化（最簡單）
        適用於 Linear, LSTM, GRU 等層
        """
        quantized_model = torch.quantization.quantize_dynamic(
            model,
            {nn.Linear, nn.LSTM, nn.GRU},  # 要量化的層類型
            dtype=torch.qint8
        )
        return quantized_model
    
    @staticmethod
    def compare_model_size(original_model, quantized_model):
        """
        比較模型大小
        """
        import tempfile
        
        # 保存原始模型
        with tempfile.NamedTemporaryFile() as f:
            torch.save(original_model.state_dict(), f.name)
            original_size = os.path.getsize(f.name) / 1024 / 1024
        
        # 保存量化模型
        with tempfile.NamedTemporaryFile() as f:
            torch.save(quantized_model.state_dict(), f.name)
            quantized_size = os.path.getsize(f.name) / 1024 / 1024
        
        print(f"模型大小比較：")
        print(f"  原始模型: {original_size:.2f} MB")
        print(f"  量化模型: {quantized_size:.2f} MB")
        print(f"  壓縮比: {original_size/quantized_size:.2f}x")
        
        return original_size, quantized_size
    
    @staticmethod
    def benchmark_quantized(original_model, quantized_model, input_shape, n_runs=100):
        """
        比較原始和量化模型的推理速度
        """
        x = torch.randn(*input_shape)
        
        # 原始模型（CPU）
        original_model = original_model.cpu().eval()
        for _ in range(10):
            _ = original_model(x)
        
        start = time.time()
        for _ in range(n_runs):
            _ = original_model(x)
        original_time = time.time() - start
        
        # 量化模型
        for _ in range(10):
            _ = quantized_model(x)
        
        start = time.time()
        for _ in range(n_runs):
            _ = quantized_model(x)
        quantized_time = time.time() - start
        
        print(f"推理速度比較 ({n_runs} 次)：")
        print(f"  原始模型: {original_time*1000:.1f} ms")
        print(f"  量化模型: {quantized_time*1000:.1f} ms")
        print(f"  加速比: {original_time/quantized_time:.2f}x")
        
        return original_time, quantized_time


# 示範動態量化
class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# 建立和量化模型
original_mlp = SimpleMLP()
quantized_mlp = ModelQuantizer.dynamic_quantize(original_mlp)

print("動態量化示範：")
ModelQuantizer.compare_model_size(original_mlp, quantized_mlp)
print()
ModelQuantizer.benchmark_quantized(original_mlp, quantized_mlp, (32, 784))

### 7.3 記憶體優化技巧

當訓練大模型時，VRAM 往往是瓶頸。以下是一些記憶體優化技巧。

---
## 🎯 總結：性能優化與部署完整知識框架

### 📊 本模組重點回顧

| 主題 | 核心技術 | 效果/使用場景 |
|------|----------|---------------|
| **DataLoader** | num_workers, pin_memory | 加速資料載入 2-5x |
| **AMP** | autocast, GradScaler | 加速訓練 2-3x, 減少 VRAM |
| **梯度累積** | 分步累積梯度 | 模擬大批次訓練 |
| **torch.compile** | 圖優化編譯 | 加速推理 10-50% |
| **量化** | INT8 量化 | 模型大小減少 4x |
| **檢查點** | 梯度檢查點 | VRAM 減少 50-80% |
| **TorchScript** | JIT 編譯 | 部署和加速 |
| **ONNX** | 跨平台格式 | 多引擎部署 |

### 🚀 性能優化決策流程

```
開始
  │
  ▼
┌─────────────────────┐
│ 1. 確認瓶頸是什麼？ │
└─────────────────────┘
  │
  ├─► 資料載入慢？
  │     └─► DataLoader 優化 (num_workers, pin_memory)
  │
  ├─► GPU 利用率低？
  │     └─► 增加 batch_size 或使用 torch.compile()
  │
  ├─► VRAM 不足？
  │     ├─► 混合精度訓練 (AMP)
  │     ├─► 梯度累積
  │     ├─► 梯度檢查點
  │     └─► 減少 batch_size
  │
  ├─► 訓練太慢？
  │     ├─► 混合精度訓練
  │     ├─► torch.compile()
  │     └─► 分散式訓練 (DDP)
  │
  └─► 推理太慢？
        ├─► TorchScript 導出
        ├─► 模型量化
        ├─► 批次推理
        └─► TensorRT 加速
```

### 🛠️ 優化技術速查表

```
┌─────────────────────────────────────────────────────────────┐
│                    訓練優化                                 │
├─────────────────────────────────────────────────────────────┤
│ 技術             │ 命令/代碼                    │ 效果     │
├─────────────────────────────────────────────────────────────┤
│ 混合精度         │ with autocast():             │ 2-3x 加速│
│ 梯度累積         │ loss = loss / accum_steps    │ 大批次   │
│ 梯度裁剪         │ clip_grad_norm_(params, 1.0) │ 穩定訓練 │
│ 學習率調度       │ CosineAnnealingLR           │ 更好收斂 │
│ 權重衰減         │ weight_decay=1e-4           │ 正則化   │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│                    推理優化                                 │
├─────────────────────────────────────────────────────────────┤
│ 技術             │ 命令/代碼                    │ 效果     │
├─────────────────────────────────────────────────────────────┤
│ torch.compile    │ model = torch.compile(model) │ 10-50% ↑ │
│ TorchScript      │ torch.jit.trace(model, x)   │ 部署用   │
│ 動態量化         │ quantize_dynamic(model)      │ 4x 壓縮  │
│ 批次推理         │ 多樣本同時推理              │ 吞吐量 ↑ │
│ ONNX Runtime     │ onnxruntime.InferenceSession│ 跨平台   │
└─────────────────────────────────────────────────────────────┘
```

### 📝 部署前 Checklist

**模型準備**：
- [ ] 模型已訓練完成並評估
- [ ] 選擇最佳檢查點
- [ ] 測試推理正確性

**優化**：
- [ ] 導出 TorchScript 或 ONNX
- [ ] 考慮量化（如果需要）
- [ ] 測試優化後的精度

**部署**：
- [ ] 建立推理服務（FastAPI / Flask）
- [ ] 設定健康檢查端點
- [ ] 實現批次推理
- [ ] 建立 Docker 容器
- [ ] 配置負載均衡（如果需要）

**監控**：
- [ ] 設定延遲監控
- [ ] 設定錯誤追蹤
- [ ] 設定模型版本管理

### 🏆 課程總結

恭喜完成整個深度學習課程！你已經掌握了：

| 模組 | 主題 | 核心技能 |
|------|------|----------|
| **00** | 環境設置 | PyTorch, CUDA, Jupyter |
| **01** | 數學基礎 | 線性代數, 微積分, 機率 |
| **02** | MLP | 前向/反向傳播, 優化器 |
| **03** | CNN | 卷積, 池化, 圖像分類 |
| **04** | RNN/LSTM | 序列建模, 梯度問題 |
| **05** | Transformer | 自注意力, 位置編碼 |
| **06** | LLM | 推理, 微調, RAG |
| **07** | VAE | 變分推理, 潛在空間 |
| **08** | GAN/Diffusion | 對抗訓練, 去噪生成 |
| **09** | GNN | 圖卷積, 訊息傳遞 |
| **10** | 部署 | 優化, 量化, 服務化 |

### 🔗 延伸學習資源

| 資源 | 說明 |
|------|------|
| [PyTorch 官方教程](https://pytorch.org/tutorials/) | 官方最佳實踐 |
| [Hugging Face Course](https://huggingface.co/course) | NLP/LLM 實務 |
| [Fast.ai](https://www.fast.ai/) | 實用深度學習 |
| [Papers With Code](https://paperswithcode.com/) | 論文 + 代碼 |
| [Weights & Biases](https://wandb.ai/site) | 實驗追蹤 |
| [MLOps Community](https://mlops.community/) | 部署最佳實踐 |

### 🎓 下一步建議

1. **選擇專精方向**
   - 電腦視覺 → 深入 CNN, ViT, 目標檢測
   - 自然語言處理 → 深入 Transformer, LLM 微調
   - 生成模型 → 深入 Diffusion, 3D 生成
   - 圖學習 → 深入 GNN, 知識圖譜

2. **實戰項目**
   - 參加 Kaggle 競賽
   - 復現經典論文
   - 建立端到端應用

3. **保持更新**
   - 關注 arXiv 最新論文
   - 追蹤 AI 領域社群
   - 參與開源項目

**感謝你完成這個課程！祝你在深度學習的旅程中一切順利！** 🚀

---
## 總結

### 本模組重點

1. **DataLoader 優化**
   - `num_workers`: 並行載入
   - `pin_memory`: 加速 CPU→GPU 傳輸
   - `prefetch_factor`: 預取批次

2. **混合精度訓練**
   - `autocast`: 自動選擇精度
   - `GradScaler`: 防止梯度下溢
   - 可提速 2-3x

3. **梯度累積**
   - 模擬大批次訓練
   - 損失除以累積步數

4. **模型保存與載入**
   - `state_dict`: 只保存權重
   - `checkpoint`: 完整訓練狀態
   - `TorchScript`: 部署用

5. **推理優化**
   - 模型融合
   - 批次處理

### 性能優化清單

```
╔══════════════════════════════════════════════════════════╗
║                    性能優化清單                          ║
╠══════════════════════════════════════════════════════════╣
║  資料載入                                                ║
║  ☐ num_workers = 4-8                                    ║
║  ☐ pin_memory = True                                    ║
║  ☐ 預處理放到 Dataset 而非訓練循環                       ║
║                                                          ║
║  訓練優化                                                ║
║  ☐ 使用混合精度 (AMP)                                   ║
║  ☐ 梯度累積（如果需要大批次）                           ║
║  ☐ 梯度裁剪（穩定訓練）                                 ║
║  ☐ 學習率調度（Cosine, OneCycle）                       ║
║                                                          ║
║  記憶體優化                                              ║
║  ☐ 梯度檢查點（大模型）                                 ║
║  ☐ 定期清理 GPU 記憶體                                  ║
║  ☐ 使用 8-bit 優化器（大模型）                          ║
║                                                          ║
║  部署優化                                                ║
║  ☐ 導出 TorchScript/ONNX                                ║
║  ☐ 批次推理                                             ║
║  ☐ 模型量化（如果需要）                                 ║
╚══════════════════════════════════════════════════════════╝
```

### 課程總結

恭喜你完成了整個深度學習課程！你已經學習了：

1. **Module 0-1**: 環境設置、數學基礎
2. **Module 2-3**: MLP、CNN 圖像分類
3. **Module 4-5**: RNN/LSTM、Transformer 序列建模
4. **Module 6**: LLM 實務應用與微調
5. **Module 7-8**: VAE、GAN、Diffusion 生成模型
6. **Module 9**: GNN 圖結構學習
7. **Module 10**: 性能優化與部署

下一步建議：
- 選擇一個感興趣的領域深入
- 參與 Kaggle 競賽實踐
- 閱讀最新論文保持更新